In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam



In [6]:

# 🔵 1️⃣ Image Processing Pipeline
def process_acne_image(image_path, output_path):
    """Process a single image and save the result."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image not found at {image_path}")
        return

    # 1️⃣ Resize image to 224x224
    image_resized = cv2.resize(image, (224, 224))

    # 2️⃣ Convert to grayscale
    gray_image = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

    # 3️⃣ Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(gray_image)

    # 4️⃣ Apply Bilateral Filter
    bilateral_filtered = cv2.bilateralFilter(clahe_image, d=9, sigmaColor=75, sigmaSpace=75)

    # 5️⃣ Apply Canny Edge Detection
    edges = cv2.Canny(bilateral_filtered, threshold1=30, threshold2=150)

    # 6️⃣ Save the processed image
    cv2.imwrite(output_path, edges)


def process_dataset_images(input_dir, output_dir):
    """Process all images in a dataset directory and save them."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        output_class_path = os.path.join(output_dir, class_folder)
        os.makedirs(output_class_path, exist_ok=True)
        
        if os.path.isdir(class_path):
            for image_filename in os.listdir(class_path):
                image_path = os.path.join(class_path, image_filename)
                output_path = os.path.join(output_class_path, image_filename)
                try:
                    process_acne_image(image_path, output_path)
                    print(f"Processed {image_path} → {output_path}")
                except Exception as e:
                    print(f"❌ Error processing {image_path}: {e}")


# Process the entire dataset
process_dataset_images('dataset/train', 'processed_dataset/train')
process_dataset_images('dataset/val', 'processed_dataset/val')
process_dataset_images('dataset/test', 'processed_dataset/test')

print("✅ Image processing complete. Processed images are stored in the 'processed_dataset' directory.")

Processed dataset/train\comedo\comedo-10.jpg → processed_dataset/train\comedo\comedo-10.jpg
Processed dataset/train\comedo\comedo-1000.jpg → processed_dataset/train\comedo\comedo-1000.jpg
Processed dataset/train\comedo\comedo-1001.jpg → processed_dataset/train\comedo\comedo-1001.jpg
Processed dataset/train\comedo\comedo-1003.jpg → processed_dataset/train\comedo\comedo-1003.jpg
Processed dataset/train\comedo\comedo-1004.jpg → processed_dataset/train\comedo\comedo-1004.jpg
Processed dataset/train\comedo\comedo-1005.jpg → processed_dataset/train\comedo\comedo-1005.jpg
Processed dataset/train\comedo\comedo-1008.jpg → processed_dataset/train\comedo\comedo-1008.jpg
Processed dataset/train\comedo\comedo-1009.jpg → processed_dataset/train\comedo\comedo-1009.jpg
Processed dataset/train\comedo\comedo-101.jpg → processed_dataset/train\comedo\comedo-101.jpg
Processed dataset/train\comedo\comedo-1010.jpg → processed_dataset/train\comedo\comedo-1010.jpg
Processed dataset/train\comedo\comedo-1011.jpg

KeyboardInterrupt: 

In [ ]:
# 🔵 2️⃣ CNN Model for Acne Detection (AcneNet)
def build_acnenet(input_shape=(224, 224, 1), num_classes=3):
    """Build the AcneNet CNN model."""
    model = Sequential([
        # Convolutional Layer 1
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        # Convolutional Layer 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        # Convolutional Layer 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        # Flatten for Fully Connected Layers
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),

        # Output Layer (3 classes: cyst, comedo, pustule)
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    return model

In [ ]:
# 🔵 3️⃣ Load the Data from the Processed Dataset
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    'processed_dataset/train/', 
    target_size=(224, 224), 
    batch_size=32, 
    color_mode='grayscale',  # Since we processed images as grayscale
    class_mode='categorical', 
    subset='training'
)

val_generator = datagen.flow_from_directory(
    'processed_dataset/train/', 
    target_size=(224, 224), 
    batch_size=32, 
    color_mode='grayscale',  # Since we processed images as grayscale
    class_mode='categorical', 
    subset='validation'
)


# 🔵 4️⃣ Build the AcneNet Model
acnenet_model = build_acnenet(input_shape=(224, 224, 1))
acnenet_model.summary()


# 🔵 5️⃣ Train the Model
history = acnenet_model.fit(
    train_generator,
    epochs=100,  # You can increase this for better results
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // 32, 
    validation_steps=val_generator.samples // 32
)





Found 5746 images belonging to 3 classes.
Found 1435 images belonging to 3 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                      

KeyboardInterrupt: 

In [ ]:
# 🔵 6️⃣ Save the Trained Model
acnenet_model.save('latest_model.h5')

print("✅ Model training complete. The model is saved as 'acne_cnn_model.h5'.")


✅ Model training complete. The model is saved as 'acne_cnn_model.h5'.
